Example Strategy

In [1]:
strategy = {
    "id": 37,
    "title": "Test A",
    "outcomes": [
        "home_win"
    ],
    "is_public": 1,
    "timer": {
        "minute": 15
    },
    "note": "",
    "hit_rate": "0.00",
    "leagues": [
        1117,
        1515,
        1103,
        1107,
        1591,
        1118,
        1108,
        711,
        1999,
        1570,
        1756,
        2004,
        1359,
        2027,
        1360,
        1361,
        1358,
        1675,
        1676,
        172,
        1373,
        1567,
        812,
        1597,
        809,
        893,
        1902,
        896,
        
    ],
    "trusted": 0,
    "active": 1,
    "strategy_prematch_rules": [
        {
            "code": "played",
            "category": "general",
            "overall": "played_overall",
            "home": "played_home",
            "away": "played_away",
            "id": 18,
            "strategy_id": 37,
            "uid": "played_general",
            "location": "overall",
            "team": "both",
            "comparator": "=",
            "value": "1.00",
            "values": [
                10,
                60
            ]
        },
        {
            "code": "home_win_probability",
            "category": "probability",
            "overall": None,
            "home": None,
            "away": None,
            "id": 19,
            "strategy_id": 37,
            "uid": "home_win_probability",
            "location": "overall",
            "team": "both",
            "comparator": "=",
            "value": "1.00",
            "values": [
                52,
                100
            ]
        },
        {
            "code": "home_win",
            "category": "odds",
            "overall": None,
            "home": None,
            "away": None,
            "id": 20,
            "strategy_id": 37,
            "uid": "home_win_odds",
            "location": "overall",
            "team": "both",
            "comparator": ">=",
            "value": "1.40",
            "values": [
                0,
                100
            ]
        }
    ],
    "strategy_inplay_rules": []
}


In [2]:

def __condition_for_probability(rule):
  return f'probability.{rule["code"]} BETWEEN {rule["values"][0]} AND {rule["values"][1]}'

def __condition_for_odds(rule):
  return f'odds.{rule["code"]} {rule["comparator"]} {rule["value"]} AND odds.type = "preodds"'

def __condition_for_team(rule, team):
  return f'{team}.{rule[rule["location"]]} BETWEEN {rule["values"][0]} AND {rule["values"][1]}'

def __condition_for_aggregate_stats(rule):
  if rule["team"] == "home":
    condition = __condition_for_team(rule, "home")
  elif rule["team"] == "away":
    condition =  __condition_for_team(rule, "away")
  else:
    condition =__condition_for_team(rule, "home") + " AND " + __condition_for_team(rule, "away")
  return condition


conditions = []
for rule in strategy["strategy_prematch_rules"]:
  if rule["category"] == "probability":
    condition = __condition_for_probability(rule)
  elif rule["category"] == "odds":
    condition = __condition_for_odds(rule)
  else:
    condition = __condition_for_aggregate_stats(rule)
  conditions.append(condition)

NameError: name 'strategy' is not defined

In [1]:
strategy_sql = "\nAND\n".join(conditions)

NameError: name 'conditions' is not defined

In [4]:
from datetime import datetime, timedelta
import time

after_24_hrs = datetime.now() + timedelta(hours=24) 
after_24_hrs_unix = time.mktime(after_24_hrs.timetuple())

In [5]:
strategy_timer = strategy["timer"]["minute"]
after_timer_minutes = datetime.now() + timedelta(minutes=strategy_timer) 
after_timer_minutes_unix = time.mktime(after_24_hrs.timetuple())

In [6]:
included_leagues = ",".join(map(str, strategy["leagues"]))
included_leagues = f"({included_leagues})"

In [8]:
print(f"""
SELECT * FROM fixtures 
WHERE 
timestamp BETWEEN {after_timer_minutes_unix} AND {after_24_hrs_unix}
AND
{strategy_sql}
AND
league_id in {included_leagues}""")


SELECT * FROM fixtures 
WHERE 
timestamp BETWEEN 1633273573.0 AND 1633273573.0
AND
home.played_overall BETWEEN 10 AND 60 AND away.played_overall BETWEEN 10 AND 60
AND
probability.home_win_probability BETWEEN 52 AND 100
AND
odds.home_win >= 1.40 AND odds.type = "preodds"
AND
league_id in (1117,1515,1103,1107,1591,1118,1108,711,1999,1570,1756,2004,1359,2027,1360,1361,1358,1675,1676,172,1373,1567,812,1597,809,893,1902,896)


In [ ]:
strategy["leagues"]

In [ ]:
import telebot

TOKEN = config["TELEGRAMBOTTOKEN"]
bot = telebot.TeleBot(TOKEN)

chat_id = 342833991

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    bot.reply_to(message, "Howdy, how are you doing?")

@bot.message_handler(func=lambda message: True)
def echo_all(message):
    bot.reply_to(message, message.text)


# bot.send_message(message.chat.id, 'You are admin of this group'!)
bot.infinity_polling()
